In [1]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)
 
os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 
import seaborn as sns

import pickle 
import logging


from config.config import SQLQuery
querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/re_decisioning
/Users/shashankgupta/Documents/code/git_project/re_decisioning/Code


In [2]:
import sys
sys.path.insert(0, '/Users/sarfrazahmed/Documents/projects/deposit-model/conf')
import pandas as pd
import numpy as np
import chart_studio.plotly as py 
from datetime import timedelta
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio
from pandas_profiling import ProfileReport
pio.templates.default = 'plotly_dark'
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
color_map = {'high': '#ef553b', 'low': '#636efa'}
import datetime
import pytz

In [ ]:
df = querySno("""with lending_data as (
select 
a.id,b.lending_business_id, a.business_id, a.novo_response_score, a.novo_risk_score, a.OFFER_AMOUNT,a.offer_start_at, a.CREATED_AT, a.status, a.reason,a.user_id
,b.novo_risk_score as novo_risk_score2, b.rejection_reasons, b.outcome,predict_meta, c.fico_score, biz.credit_limit, biz.interest_rate, biz.remaining_limit
from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_INVITATIONS a
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_BUSINESSES" biz
on a.business_id = biz.business_id
left join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS b 
on a.business_id=json_extract_path_text(b.predict_meta, 'business_id')
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."CUSTOMER_CREDIT_PROFILES" c
on a.user_id = c.user_id
where (a.business_id != '' and a.business_id is not null)
)


,bins as (
select lending_data.*,
case 
        when novo_response_score >= 546 then 5
        when novo_response_score >= 405 and novo_response_score < 546 then 4
        when novo_response_score >= 302 and novo_response_score < 405 then 3
        when novo_response_score >= 184 and novo_response_score < 302 then 2
        when novo_response_score < 184 then 1
    else null end as response_bin,
    
    case 
        when novo_risk_score >= 656 then 5
        when novo_risk_score >= 545 and novo_risk_score < 656 then 4
        when novo_risk_score >= 489 and novo_risk_score < 545 then 3
        when novo_risk_score >= 443 and novo_risk_score < 489 then 2
        when novo_risk_score < 443 then 1
    else null end as rs1_bin, 
    json_extract_path_text(predict_meta, 'default_bin') as rs2_bin,
    case when fico_score >= 800 then 5 
    when fico_score >= 750 then 4 
    when fico_score >= 700 then 3 
    when fico_score >= 650 then 2 
    when fico_score >= 600 then 1 end as fico_bin
    -- count(business_id) cnt
    -- avg(fico_score) as avg_fico_score 
    from lending_data
    where novo_risk_score2 is not null 
    and rejection_reasons is not null
)

select * from bins
 """)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['credit_limit'] = df['credit_limit']/100

In [ ]:
df['rs2_bin'] = df['rs2_bin'].astype('int')

In [ ]:
# Median - credit limit
df.groupby(['rs2_bin'],as_index=False)['credit_limit'].median().sort_values(by='rs2_bin',ascending=False)

In [ ]:
# Mean - Credit limit
df.groupby(['rs2_bin'],as_index=False)['credit_limit'].mean().sort_values(by='rs2_bin',ascending=False)

In [ ]:
# Revenue estimates for jan aged accounts
rev_df = querySno(""" 
with min_dates as (
select lending_business_id, min(created_at) as min_created_at, min(due_date) as min_due_date from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES"
group by 1
)
, jan_aged_acc as (
-- select sum(fpd), count(fpd) as aged_accounts from (
select distinct a.lending_business_id, case when paid_on_time = 'false' then 1 else 0 end as fpd
from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES" a 
left join min_dates b on a.lending_business_id = b.lending_business_id 
where to_date(b.min_created_at) > to_date('2023-01-02')
and to_date(b.min_due_date) <= to_date(getdate())
and to_date(a.due_date) <= to_date(getdate())
-- )
)


select a.*,c.* from jan_aged_acc a
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_BUSINESSES" b
on a.lending_business_id = b.id
left join "PROD_DB"."ADHOC"."LENDING_REVENUE_ESTIMATION_V2" c
on b.business_id = c.business_id


""")

In [ ]:
rev_df.shape

In [ ]:
rev_df.head()

In [ ]:
#FPD rate
rev_df['fpd'].sum()/len(rev_df)

In [ ]:
rev_df['avg_predicted_revenue'].mean()

In [ ]:
rev_df['past_1month_rev'].mean()

In [ ]:
rev_df.loc[rev_df['fpd']==1 , 'past_1month_rev'].median()

In [ ]:
rev_df.loc[rev_df['fpd']==0 , 'past_1month_rev'].median()

#### With Outliers

In [ ]:
fig = px.histogram(rev_df,x='past_1month_rev',color = 'fpd',text_auto=True,nbins=10)
fig.show()

In [ ]:
fig = px.histogram(rev_df,x='past_1month_rev',color = 'fpd',text_auto=True,barnorm='percent',nbins=5).update_layout(
                                             yaxis_title="Percentage",
                                             yaxis={"ticksuffix":"%"})
fig.show()

#### Excluding Outliers

In [ ]:
df2 = rev_df[rev_df['past_1month_rev']<70000]

In [ ]:
df2.shape

In [ ]:
fig = px.histogram(df2,x='past_1month_rev',color = 'fpd',text_auto=True,barnorm='percent',nbins=5).update_layout(
                                             yaxis_title="Percentage",
                                             yaxis={"ticksuffix":"%"})
fig.show()

In [ ]:
fig = px.histogram(df2,x='past_1month_rev',color = 'fpd',text_auto=True,nbins=5)
fig.show()

In [ ]:
#rev inc trend - 3 months
df2[(df2['past_1month_rev']>df2['past_2month_rev']) & (df2['past_2month_rev']>df2['past_3month_rev'])].shape

In [ ]:
len(df2[df2['past_1month_rev']>df2['past_2month_rev']])

##### Creating flags

In [ ]:
df2['rev_inc_3mths'] = df2.apply(lambda x : 1 if (x['past_1month_rev']>x['past_2month_rev']) & (x['past_2month_rev']>x['past_3month_rev']) else 0, axis=1)

In [ ]:
df2['rev_inc_3mths'].sum()

In [ ]:
df2['past1_grt_past2'] = df2.apply(lambda x : 1 if x['past_1month_rev'] > x['past_2month_rev'] else 0, axis=1)

In [ ]:
df2['past1_grt_past2'].sum()

In [ ]:
df2['predicted_grt_past1'] = df2.apply(lambda x : 1 if x['avg_predicted_revenue'] > x['past_1month_rev'] else 0, axis=1)

In [ ]:
df2['predicted_grt_past1'].sum()

##### Comparing FPDs

In [ ]:
fig = px.histogram(df2,x='rev_inc_3mths',color = 'fpd',text_auto=True)
fig.show()

In [ ]:
fig = px.histogram(df2,x='rev_inc_3mths',color = 'fpd',text_auto=True,barnorm='percent').update_layout(
                                             yaxis_title="Percentage",
                                             yaxis={"ticksuffix":"%"})
fig.show()

In [ ]:
fig = px.histogram(df2,x='past1_grt_past2',color = 'fpd',text_auto=True)
fig.show()

In [ ]:
fig = px.histogram(df2,x='past1_grt_past2',color = 'fpd',text_auto=True,barnorm='percent').update_layout(
                                             yaxis_title="Percentage",
                                             yaxis={"ticksuffix":"%"})
fig.show()

In [ ]:
fig = px.histogram(df2,x='predicted_grt_past1',color = 'fpd',text_auto=True,barnorm='percent').update_layout(
                                             yaxis_title="Percentage",
                                             yaxis={"ticksuffix":"%"})
fig.show()

#### At offer Start

In [ ]:
# At offer start
querySno = SQLQuery('snowflake')
rev_df1 = querySno(r"""
with lending_data as (
select 
a.id,b.lending_business_id, a.business_id, a.OFFER_AMOUNT,a.offer_start_at, a.CREATED_AT, a.user_id,pay.first_due_date,
biz.credit_limit, biz.interest_rate, biz.remaining_limit
from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_INVITATIONS a
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_BUSINESSES" biz
on a.business_id = biz.business_id
left join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS b 
on a.business_id=json_extract_path_text(b.predict_meta, 'business_id')
left join (select LENDING_BUSINESS_ID, min(created_at) as min_created_at, min(due_date) as first_due_date from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES"
group by 1) pay
on b.lending_business_id = pay.lending_business_id
where (a.business_id != '' and a.business_id is not null)
),

external_active_account_names AS (
SELECT t1.id AS plaid_item_id, t1.business_id, lower(regexp_replace(t2.value, '(inc|inc.|llc|\\,|\\.|co|corp.|co.|corp)', '', 1,0,'i')) AS external_acc_company_name  
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC.plaid_items t1, LATERAL FLATTEN(input => "IDENTITY":names) t2
WHERE status = 'active'
),
target_businesses AS (
SELECT DISTINCT a.id AS business_id, a.TYPE AS business_type, 
               lower(regexp_replace(a.company_name, '(inc|inc.|llc|\\,|\\.|co|corp.|co.|corp)', '', 1,0,'i')) AS company_name, 
               CASE WHEN dba IS NOT NULL THEN lower(regexp_replace(dba, '(inc|inc.|llc|\\,|\\.|co|corp.|co.|corp)', '', 1,0,'i')) ELSE NULL end AS dba 
FROM  FIVETRAN_DB.PROD_NOVO_API_PUBLIC.BUSINESSES a INNER JOIN  FIVETRAN_DB.PROD_NOVO_API_PUBLIC.PLAID_TRANSACTIONS b ON a.id=b.business_id
inner join PROD_DB.DATA.BUSINESSES biz on a.id=biz.business_id
inner join lending_data d on a.id = d.business_id
WHERE a.status='active' AND biz.IS_ACCOUNT_FUNDED = true 
AND  datediff(month, biz.ACCOUNT_FUNDED_AT, d.offer_start_at) <= 6
),
extrenal_accounts_non_sol_prop AS (
SELECT c.business_id, a.external_acc_company_name, 
              b.EXTERNAL_ACCOUNT_ID, business_type, company_name  
              FROM external_active_account_names a
			  LEFT JOIN FIVETRAN_DB.PROD_NOVO_API_PUBLIC.PLAID_EXTERNAL_ACCOUNTS b ON a.plaid_item_id = b.PLAID_ITEM_ID 
			  LEFT JOIN target_businesses c ON a.BUSINESS_ID  = c.business_id
WHERE c.business_type != 'sole_proprietorship'  
			  AND ( (company_name = external_acc_company_name) OR 
			              (SOUNDEX(company_name) = SOUNDEX(external_acc_company_name)) OR
			              (dba IS NOT NULL AND dba = external_acc_company_name) OR
			              (dba IS NOT NULL AND SOUNDEX(dba)=SOUNDEX(external_acc_company_name))
			             )
			  AND c.business_id IN (SELECT business_id FROM lending_data)
),
extrenal_non_sol_prop_revenue AS (
SELECT a.business_id, amount,TRANSACTION_id, DATEDIFF(month, a.DATE, d.offer_start_at) AS txn_month, 
          CASE WHEN CATEGORIES ILIKE '%Insurance%' and TYPE = 'credit' THEN 1 ELSE 0 END as INSURANCE_CREDIT_TXN,
          CASE WHEN CATEGORIES ILIKE '%Loan%' and TYPE = 'credit' THEN 1 ELSE 0 END as LOAN_CREDIT_TXN,
          CASE WHEN CATEGORIES ILIKE '%Tax%' AND CATEGORIES NOT ILIKE '%Taxi%' and TYPE = 'credit' THEN 1 ELSE 0 END as TAX_CREDIT_TXN,
          CASE WHEN CATEGORIES ILIKE '%Insurance%' and TYPE = 'credit' THEN AMOUNT ELSE 0 END as INSURANCE_CREDIT_AMT,
          CASE WHEN CATEGORIES ILIKE '%Loan%' and TYPE = 'credit' THEN AMOUNT ELSE 0 END as LOAN_CREDIT_AMT,
          CASE WHEN CATEGORIES ILIKE '%Tax%' AND CATEGORIES NOT ILIKE '%Taxi%' and TYPE = 'credit' THEN AMOUNT ELSE 0 END as TAX_CREDIT_AMT
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC.PLAID_TRANSACTIONS  a INNER JOIN extrenal_accounts_non_sol_prop b
ON a.EXTERNAL_ACCOUNT_ID = b.EXTERNAL_ACCOUNT_ID
inner join lending_data d on a.business_id = d.business_id
WHERE  datediff(MONTH, a.DATE, d.offer_start_at) <= 6 and datediff(MONTH, a.DATE, d.offer_start_at) > 0
       and a.pending = FALSE AND a.TYPE = 'credit'
),
all_credits_plaid as(
SELECT BUSINESS_ID, txn_month, sum(-amount)  AS total_credit_amount_plaid, 
               sum(-LOAN_CREDIT_AMT) AS total_loan_credit_amt_plaid, 
               sum(-INSURANCE_CREDIT_AMT) AS total_insurance_credit_amt_plaid, 
               sum(-TAX_CREDIT_AMT) AS total_tax_credit_amt_plaid
FROM extrenal_non_sol_prop_revenue 
GROUP BY BUSINESS_ID, txn_month
),
txns_with_categories AS (
select a.business_id, DATEDIFF(month, TRANSACTION_DATE, d.offer_start_at) AS txn_month, amount, description,
			REGEXP_REPLACE(REGEXP_REPLACE(lower(description), ':',' '), '[^a-z ]', '') AS description_processed, transaction_date,
			            CASE WHEN description_processed ILIKE any(
                                                '%novo funding%',
			            						'% earninactivehours %', 
                                                'earninactivehours %',
                                                '% earninactivehours',
                                                'earninactivehours',
			            						'% activehours %', 
                                                'activehours %', 
                                                '% activehours', 
                                                'activehours', 
			            						'% navchek %', 
                                                'navchek %', 
                                                '% navchek', 
                                                'navchek', 
			            						'% loan %', 
                                                'loan %',
                                                '% loan',
                                                'loan',
			            						'% earnin %',
                                                'earnin %',
                                                '% earnin',
                                                'earnin', 
			            						'% advance %',
                                                '% advance',
                                                'advance %',
                                                'advance'
			            ) THEN 1 ELSE 0 END AS LOANS,
			            CASE WHEN description_processed ILIKE any(
			            						'% hcclaimpmt %',
                                                '% hcclaimpmt',
                                                'hcclaimpmt %',
                                                'hcclaimpmt',
												 '% unitedhealthcare %',
                                                 'unitedhealthcare %',
                                                 '% unitedhealthcare',
                                                 'unitedhealthcare',
												 '% bcbs %',
                                                 '% bcbs',
                                                 'bcbs %',
                                                 'bcbs',
												 '% cigna %',
                                                 'cigna %',
                                                 '% cigna',
                                                 'cigna',
												 '% govt %',
                                                 '% govt',
                                                 'govt %',
                                                 'govt',
												 '% humana %',
                                                 '% humana',
                                                 'humana %',
                                                 'humana',
												 '% aetna %',
                                                 'aetna %',
                                                 '% aetna',
                                                 'aetna',
												 '% insurance %',
                                                 'insurance %',
                                                 '% insurance',
                                                 'insurance',
												 '% life %',
                                                 'life %',
                                                 '% life',
                                                 'life',
												 '% blue %',
                                                 '% blue',
                                                 'blue %',
                                                 'blue'
						) THEN 1 ELSE 0 END AS INSURANCE,
						CASE WHEN description_processed ILIKE any(
												'% sbtpg %',
                                                '% sbtpg',
                                                'sbtpg %',
                                                'sbtpg', 
												'% xxtaxeip %',
                                                'xxtaxeip %',
                                                '% xxtaxeip',
                                                'xxtaxeip', 
												'% tax %', 
                                                'tax %', 
                                                '% tax', 
                                                'tax', 
												'% irs %', 
                                                '% irs', 
                                                'irs %', 
                                                'irs', 
												'% treas %', 
                                                '% treas', 
                                                'treas %', 
                                                'treas', 
												'% taxrfd %',
                                                '% taxrfd',
                                                'taxrfd %',
                                                'taxrfd'
						) THEN 1 ELSE 0 END AS TAX
from fivetran_db.PROD_NOVO_API_PUBLIC.transactions a inner join lending_data d on a.business_id = d.business_id
		   WHERE datediff(MONTH ,TRANSACTION_DATE,d.offer_start_at) <= 6 and datediff(MONTH ,TRANSACTION_DATE,d.offer_start_at) > 0
           AND status='active' AND TYPE = 'credit'     
)
,all_credits AS (
SELECT a.BUSINESS_ID, txn_month,
               sum(amount)  AS total_credit_amount, 
               sum(CASE WHEN LOANS = 1 THEN amount ELSE 0 end) AS total_loan_credit_amount, 
               sum(CASE WHEN INSURANCE = 1 THEN amount ELSE 0 END) AS total_insurance_credit_amount, 
               sum(CASE WHEN TAX = 1 THEN amount ELSE 0 end) AS total_tax_credit_amount
FROM  txns_with_categories a
GROUP BY a.BUSINESS_ID, txn_month
),
all_txn_credits AS (
SELECT business_id, txn_month, 
               COALESCE(total_credit_amount, 0) AS total_credit_amount,
               COALESCE(total_loan_credit_amount, 0) AS total_loan_credit_amount,
               COALESCE(total_insurance_credit_amount, 0) AS total_insurance_credit_amount,
               COALESCE(total_tax_credit_amount, 0) AS total_tax_credit_amount,
               COALESCE(TOTAL_CREDIT_AMOUNT_PLAID, 0) AS TOTAL_CREDIT_AMOUNT_PLAID,
               COALESCE(TOTAL_LOAN_CREDIT_AMT_PLAID, 0) AS TOTAL_LOAN_CREDIT_AMT_PLAID,
               COALESCE(TOTAL_INSURANCE_CREDIT_AMT_PLAID, 0) AS TOTAL_INSURANCE_CREDIT_AMT_PLAID,
               COALESCE(TOTAL_TAX_CREDIT_AMT_PLAID, 0) AS TOTAL_TAX_CREDIT_AMT_PLAID
FROM all_credits LEFT JOIN all_credits_plaid using(business_id, txn_month)
)
,last_6mnth_revenue as (
SELECT business_id, 
       CASE WHEN txn_month = 1 THEN 'past_1month_rev'
            WHEN txn_month = 2 THEN 'past_2month_rev'
            WHEN txn_month = 3 THEN 'past_3month_rev'
            WHEN txn_month in (4,5,6) THEN 'past_furthest_3months'
       END as month,
       ((total_credit_amount-total_loan_credit_amount-total_insurance_credit_amount-total_tax_credit_amount)
	           + (TOTAL_CREDIT_AMOUNT_PLAID - TOTAL_LOAN_CREDIT_AMT_PLAID - TOTAL_INSURANCE_CREDIT_AMT_PLAID - TOTAL_TAX_CREDIT_AMT_PLAID)) AS revenue
FROM all_txn_credits
where txn_month <= 6)

,pivot_revenue as (
select * from last_6mnth_revenue
    pivot (sum(revenue) for month in ('past_1month_rev', 'past_2month_rev', 'past_3month_rev', 'past_furthest_3months'))
      as p (business_id, past_1month_rev, past_2month_rev, past_3month_rev, past_furthest_3months)

)

select *, 
       ZEROIFNULL(past_1month_rev)*0.1508
      +ZEROIFNULL(past_2month_rev)*0.1154
      +ZEROIFNULL(past_3month_rev)*0.0854
      +ZEROIFNULL(past_furthest_3months/3)*0.2136
      +460.6110
      as avg_predicted_revenue,
      6*avg_predicted_revenue AS predicted_revenue_next_6mo
from pivot_revenue
"""
)

In [ ]:
rev_df1.shape

In [ ]:
rev_df1.head()

#### At first due date

In [ ]:
# At first due date
querySno = SQLQuery('snowflake')
rev_df2 = querySno(r"""
with lending_data as (
select 
a.id,b.lending_business_id, a.business_id, a.OFFER_AMOUNT,a.offer_start_at, a.CREATED_AT, a.user_id,pay.first_due_date,
biz.credit_limit, biz.interest_rate, biz.remaining_limit
from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_INVITATIONS a
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_BUSINESSES" biz
on a.business_id = biz.business_id
left join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS b 
on a.business_id=json_extract_path_text(b.predict_meta, 'business_id')
left join (select LENDING_BUSINESS_ID, min(created_at) as min_created_at, min(due_date) as first_due_date from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES"
group by 1) pay
on b.lending_business_id = pay.lending_business_id
where (a.business_id != '' and a.business_id is not null)
),

external_active_account_names AS (
SELECT t1.id AS plaid_item_id, t1.business_id, lower(regexp_replace(t2.value, '(inc|inc.|llc|\\,|\\.|co|corp.|co.|corp)', '', 1,0,'i')) AS external_acc_company_name  
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC.plaid_items t1, LATERAL FLATTEN(input => "IDENTITY":names) t2
WHERE status = 'active'
),
target_businesses AS (
SELECT DISTINCT a.id AS business_id, a.TYPE AS business_type, 
               lower(regexp_replace(a.company_name, '(inc|inc.|llc|\\,|\\.|co|corp.|co.|corp)', '', 1,0,'i')) AS company_name, 
               CASE WHEN dba IS NOT NULL THEN lower(regexp_replace(dba, '(inc|inc.|llc|\\,|\\.|co|corp.|co.|corp)', '', 1,0,'i')) ELSE NULL end AS dba 
FROM  FIVETRAN_DB.PROD_NOVO_API_PUBLIC.BUSINESSES a INNER JOIN  FIVETRAN_DB.PROD_NOVO_API_PUBLIC.PLAID_TRANSACTIONS b ON a.id=b.business_id
inner join PROD_DB.DATA.BUSINESSES biz on a.id=biz.business_id
inner join lending_data d on a.id = d.business_id
WHERE a.status='active' AND biz.IS_ACCOUNT_FUNDED = true 
AND  datediff(month, biz.ACCOUNT_FUNDED_AT, d.first_due_date) <= 6
),
extrenal_accounts_non_sol_prop AS (
SELECT c.business_id, a.external_acc_company_name, 
              b.EXTERNAL_ACCOUNT_ID, business_type, company_name  
              FROM external_active_account_names a
			  LEFT JOIN FIVETRAN_DB.PROD_NOVO_API_PUBLIC.PLAID_EXTERNAL_ACCOUNTS b ON a.plaid_item_id = b.PLAID_ITEM_ID 
			  LEFT JOIN target_businesses c ON a.BUSINESS_ID  = c.business_id
WHERE c.business_type != 'sole_proprietorship'  
			  AND ( (company_name = external_acc_company_name) OR 
			              (SOUNDEX(company_name) = SOUNDEX(external_acc_company_name)) OR
			              (dba IS NOT NULL AND dba = external_acc_company_name) OR
			              (dba IS NOT NULL AND SOUNDEX(dba)=SOUNDEX(external_acc_company_name))
			             )
			  AND c.business_id IN (SELECT business_id FROM lending_data)
),
extrenal_non_sol_prop_revenue AS (
SELECT a.business_id, amount,TRANSACTION_id, DATEDIFF(month, a.DATE, d.first_due_date) AS txn_month, 
          CASE WHEN CATEGORIES ILIKE '%Insurance%' and TYPE = 'credit' THEN 1 ELSE 0 END as INSURANCE_CREDIT_TXN,
          CASE WHEN CATEGORIES ILIKE '%Loan%' and TYPE = 'credit' THEN 1 ELSE 0 END as LOAN_CREDIT_TXN,
          CASE WHEN CATEGORIES ILIKE '%Tax%' AND CATEGORIES NOT ILIKE '%Taxi%' and TYPE = 'credit' THEN 1 ELSE 0 END as TAX_CREDIT_TXN,
          CASE WHEN CATEGORIES ILIKE '%Insurance%' and TYPE = 'credit' THEN AMOUNT ELSE 0 END as INSURANCE_CREDIT_AMT,
          CASE WHEN CATEGORIES ILIKE '%Loan%' and TYPE = 'credit' THEN AMOUNT ELSE 0 END as LOAN_CREDIT_AMT,
          CASE WHEN CATEGORIES ILIKE '%Tax%' AND CATEGORIES NOT ILIKE '%Taxi%' and TYPE = 'credit' THEN AMOUNT ELSE 0 END as TAX_CREDIT_AMT
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC.PLAID_TRANSACTIONS  a INNER JOIN extrenal_accounts_non_sol_prop b
ON a.EXTERNAL_ACCOUNT_ID = b.EXTERNAL_ACCOUNT_ID
inner join lending_data d on a.business_id = d.business_id
WHERE  datediff(MONTH, a.DATE, d.first_due_date) <= 6 and datediff(MONTH, a.DATE, d.first_due_date) > 0
       and a.pending = FALSE AND a.TYPE = 'credit'
),
all_credits_plaid as(
SELECT BUSINESS_ID, txn_month, sum(-amount)  AS total_credit_amount_plaid, 
               sum(-LOAN_CREDIT_AMT) AS total_loan_credit_amt_plaid, 
               sum(-INSURANCE_CREDIT_AMT) AS total_insurance_credit_amt_plaid, 
               sum(-TAX_CREDIT_AMT) AS total_tax_credit_amt_plaid
FROM extrenal_non_sol_prop_revenue 
GROUP BY BUSINESS_ID, txn_month
),
txns_with_categories AS (
select a.business_id, DATEDIFF(month, TRANSACTION_DATE, d.first_due_date) AS txn_month, amount, description,
			REGEXP_REPLACE(REGEXP_REPLACE(lower(description), ':',' '), '[^a-z ]', '') AS description_processed, transaction_date,
			            CASE WHEN description_processed ILIKE any(
                                                '%novo funding%',
			            						'% earninactivehours %', 
                                                'earninactivehours %',
                                                '% earninactivehours',
                                                'earninactivehours',
			            						'% activehours %', 
                                                'activehours %', 
                                                '% activehours', 
                                                'activehours', 
			            						'% navchek %', 
                                                'navchek %', 
                                                '% navchek', 
                                                'navchek', 
			            						'% loan %', 
                                                'loan %',
                                                '% loan',
                                                'loan',
			            						'% earnin %',
                                                'earnin %',
                                                '% earnin',
                                                'earnin', 
			            						'% advance %',
                                                '% advance',
                                                'advance %',
                                                'advance'
			            ) THEN 1 ELSE 0 END AS LOANS,
			            CASE WHEN description_processed ILIKE any(
			            						'% hcclaimpmt %',
                                                '% hcclaimpmt',
                                                'hcclaimpmt %',
                                                'hcclaimpmt',
												 '% unitedhealthcare %',
                                                 'unitedhealthcare %',
                                                 '% unitedhealthcare',
                                                 'unitedhealthcare',
												 '% bcbs %',
                                                 '% bcbs',
                                                 'bcbs %',
                                                 'bcbs',
												 '% cigna %',
                                                 'cigna %',
                                                 '% cigna',
                                                 'cigna',
												 '% govt %',
                                                 '% govt',
                                                 'govt %',
                                                 'govt',
												 '% humana %',
                                                 '% humana',
                                                 'humana %',
                                                 'humana',
												 '% aetna %',
                                                 'aetna %',
                                                 '% aetna',
                                                 'aetna',
												 '% insurance %',
                                                 'insurance %',
                                                 '% insurance',
                                                 'insurance',
												 '% life %',
                                                 'life %',
                                                 '% life',
                                                 'life',
												 '% blue %',
                                                 '% blue',
                                                 'blue %',
                                                 'blue'
						) THEN 1 ELSE 0 END AS INSURANCE,
						CASE WHEN description_processed ILIKE any(
												'% sbtpg %',
                                                '% sbtpg',
                                                'sbtpg %',
                                                'sbtpg', 
												'% xxtaxeip %',
                                                'xxtaxeip %',
                                                '% xxtaxeip',
                                                'xxtaxeip', 
												'% tax %', 
                                                'tax %', 
                                                '% tax', 
                                                'tax', 
												'% irs %', 
                                                '% irs', 
                                                'irs %', 
                                                'irs', 
												'% treas %', 
                                                '% treas', 
                                                'treas %', 
                                                'treas', 
												'% taxrfd %',
                                                '% taxrfd',
                                                'taxrfd %',
                                                'taxrfd'
						) THEN 1 ELSE 0 END AS TAX
from fivetran_db.PROD_NOVO_API_PUBLIC.transactions a inner join lending_data d on a.business_id = d.business_id
		   WHERE datediff(MONTH ,TRANSACTION_DATE,d.first_due_date) <= 6 and datediff(MONTH ,TRANSACTION_DATE,d.first_due_date) > 0
           AND status='active' AND TYPE = 'credit'     
)
,all_credits AS (
SELECT a.BUSINESS_ID, txn_month,
               sum(amount)  AS total_credit_amount, 
               sum(CASE WHEN LOANS = 1 THEN amount ELSE 0 end) AS total_loan_credit_amount, 
               sum(CASE WHEN INSURANCE = 1 THEN amount ELSE 0 END) AS total_insurance_credit_amount, 
               sum(CASE WHEN TAX = 1 THEN amount ELSE 0 end) AS total_tax_credit_amount
FROM  txns_with_categories a
GROUP BY a.BUSINESS_ID, txn_month
),
all_txn_credits AS (
SELECT business_id, txn_month, 
               COALESCE(total_credit_amount, 0) AS total_credit_amount,
               COALESCE(total_loan_credit_amount, 0) AS total_loan_credit_amount,
               COALESCE(total_insurance_credit_amount, 0) AS total_insurance_credit_amount,
               COALESCE(total_tax_credit_amount, 0) AS total_tax_credit_amount,
               COALESCE(TOTAL_CREDIT_AMOUNT_PLAID, 0) AS TOTAL_CREDIT_AMOUNT_PLAID,
               COALESCE(TOTAL_LOAN_CREDIT_AMT_PLAID, 0) AS TOTAL_LOAN_CREDIT_AMT_PLAID,
               COALESCE(TOTAL_INSURANCE_CREDIT_AMT_PLAID, 0) AS TOTAL_INSURANCE_CREDIT_AMT_PLAID,
               COALESCE(TOTAL_TAX_CREDIT_AMT_PLAID, 0) AS TOTAL_TAX_CREDIT_AMT_PLAID
FROM all_credits LEFT JOIN all_credits_plaid using(business_id, txn_month)
)
,last_6mnth_revenue as (
SELECT business_id, 
       CASE WHEN txn_month = 1 THEN 'past_1month_rev'
            WHEN txn_month = 2 THEN 'past_2month_rev'
            WHEN txn_month = 3 THEN 'past_3month_rev'
            WHEN txn_month in (4,5,6) THEN 'past_furthest_3months'
       END as month,
       ((total_credit_amount-total_loan_credit_amount-total_insurance_credit_amount-total_tax_credit_amount)
	           + (TOTAL_CREDIT_AMOUNT_PLAID - TOTAL_LOAN_CREDIT_AMT_PLAID - TOTAL_INSURANCE_CREDIT_AMT_PLAID - TOTAL_TAX_CREDIT_AMT_PLAID)) AS revenue
FROM all_txn_credits
where txn_month <= 6)

,pivot_revenue as (
select * from last_6mnth_revenue
    pivot (sum(revenue) for month in ('past_1month_rev', 'past_2month_rev', 'past_3month_rev', 'past_furthest_3months'))
      as p (business_id, past_1month_rev, past_2month_rev, past_3month_rev, past_furthest_3months)

)

select *, 
       ZEROIFNULL(past_1month_rev)*0.1508
      +ZEROIFNULL(past_2month_rev)*0.1154
      +ZEROIFNULL(past_3month_rev)*0.0854
      +ZEROIFNULL(past_furthest_3months/3)*0.2136
      +460.6110
      as avg_predicted_revenue,
      6*avg_predicted_revenue AS predicted_revenue_next_6mo
from pivot_revenue
"""
)

In [ ]:
rev_df2.shape

In [ ]:
rev_df2.head()

In [ ]:
mdf = rev_df2.merge(rev_df1,how='inner',left_on='business_id',right_on='business_id')

In [ ]:
mdf.head()

#### FPD

In [ ]:
querySno = SQLQuery('snowflake')
fpd = querySno(r"""
with min_dates as (
select lending_business_id, min(created_at) as min_created_at, min(due_date) as first_due_date from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES"
group by 1
)

,fpd_flag as 
(
select lending_business_id, Loan_month, max(fpd) fpd from (
select  a.lending_business_id, case when paid_on_time = 'false' then 1 else 0 end as fpd, 
case when month(d.created_at) = 1 then 'Jan'
when month(d.created_at) = 2 then 'Feb' else 'Other' end as Loan_month
from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES" a 
left join min_dates b on a.lending_business_id = b.lending_business_id
left join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS d on d.lending_business_id = a.lending_business_id
where to_date(b.min_created_at) > to_date('2023-01-02')
and to_date(a.due_date) = to_date(b.first_due_date)
and to_date(b.first_due_date) <= to_date(getdate())
)
group by 1,2
)

, lending_data as (
select 
a.id,b.lending_business_id, a.business_id, a.novo_response_score, a.novo_risk_score, a.CREATED_AT, a.status, a.reason,a.user_id
,b.novo_risk_score as novo_risk_score2, b.rejection_reasons, b.outcome,predict_meta, c.fico_score
from (select * from 
(select *, row_number() over(partition by business_id order by offer_start_at desc) rnk from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_INVITATIONS)
where rnk = 1) a
left join
FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS b 
on a.business_id=json_extract_path_text(b.predict_meta, 'business_id')
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."CUSTOMER_CREDIT_PROFILES" c
on a.user_id = c.user_id or b.lending_business_id = c.lending_business_id
where (a.business_id != '' and a.business_id is not null)
)


,bins as (
select lending_data.*,
case 
        when novo_response_score >= 546 then 5
        when novo_response_score >= 405 and novo_response_score < 546 then 4
        when novo_response_score >= 302 and novo_response_score < 405 then 3
        when novo_response_score >= 184 and novo_response_score < 302 then 2
        when novo_response_score < 184 then 1
    else null end as response_bin,
    
    case 
        when novo_risk_score >= 656 then 5
        when novo_risk_score >= 545 and novo_risk_score < 656 then 4
        when novo_risk_score >= 489 and novo_risk_score < 545 then 3
        when novo_risk_score >= 443 and novo_risk_score < 489 then 2
        when novo_risk_score < 443 then 1
    else null end as rs1_bin, 
    json_extract_path_text(predict_meta, 'default_bin') as rs2_bin,
    -- case when fico_score >= 800 then '>800'
    -- when fico_score >= 750 then
    case when fico_score >= 700 then '>700'
    when fico_score >= 650 then '650-700'
    when fico_score >= 600 then '600-650' 
    else '<600' end as fico_bin
    -- count(business_id) cnt
    -- avg(fico_score) as avg_fico_score 
    from lending_data
    -- where novo_risk_score2 is not null 
    -- and rejection_reasons is not null
)

select a.lending_business_id,b.business_id,loan_month, response_bin, rs1_bin, rs2_bin,fico_bin,fico_score, fpd from fpd_flag a 
left join bins b 
on a.lending_business_id = b.lending_business_id
--update to remove pilot migrators
join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_INVITATIONS i
on i.business_id = b.business_id
where i.segment != 'Pilot migration'

""")

In [ ]:
fpd.shape

In [ ]:
fpd.head()

In [ ]:
rev_df = mdf[['business_id','past_1month_rev_x','avg_predicted_revenue_y']].rename(columns={'past_1month_rev_x':'1st_month_rev_actual', 'avg_predicted_revenue_y':'avg_predicted_rev'})


In [ ]:
rev_df.head()

In [ ]:
rev_df['ratio_actual_pred_rev'] = rev_df['1st_month_rev_actual']/rev_df['avg_predicted_rev']

In [ ]:
rev_df = rev_df.merge(fpd,how='inner',left_on='business_id',right_on='business_id')

In [ ]:
rev_df.shape

In [ ]:
rev_df['fpd'].sum()

In [ ]:
rev_df[rev_df['ratio_actual_pred_rev']<1].shape

In [ ]:
rev_df.loc[rev_df['ratio_actual_pred_rev']<1,'fpd'].sum()

In [ ]:
rev_df[rev_df['ratio_actual_pred_rev']>=1].shape

In [ ]:
rev_df.loc[rev_df['ratio_actual_pred_rev']>=1,'fpd'].sum()

In [ ]:
rev_df['ratio_actual_pred_rev'].describe()

In [ ]:
fig = px.histogram(rev_df,x='ratio_actual_pred_rev',color = 'fpd',text_auto=True)
fig.show()

#### Lending features

In [6]:
querySno = SQLQuery('snowflake')
lending_features = r"""
WITH past_6_mo_txn as (
  		SELECT t.id AS transaction_id, t.BUSINESS_ID , amount, TYPE, TRANSACTION_DATE,  t.status, running_balance,
        pay.lending_business_id, pay.min_created_at, pay.first_due_date,
              CASE WHEN TYPE = 'credit' and DESCRIPTION  ILIKE ANY(
	                    '%AMAZON%'
	                    , '%AMZN%'
	                    , '%STRIPE%'
	                    , '%SQUARE INC%'
	                    , '%SHOPIFY%'
	                    , '%SHOPPAY%'
	                    )
                    AND NOT(
	                    description ILIKE ANY(
	                        '%LYFT'
	                        , '%OfferUp%'
	                        , '%Gumroad%'
	                        , '%FB%Fundrai%'
	                        , '%Verify%'
	                        , '%CASH%'
	                        , '%PAYROLL%'
	                        , '%VRFY%'
	                        , '%CAPITAL%'
	                        , '%REFUND%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
                CASE WHEN TYPE ='debit' AND MEDIUM  ILIKE ANY('%external withdrawal%', '%iat withdrawal%') THEN 1 ELSE 0 END AS ach_debit_txn,
                CASE WHEN type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') THEN 1 ELSE 0 END AS ach_credit_txn,
                CASE WHEN medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
                CASE WHEN TYPE = 'credit' AND description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,
                datediff(day, TRANSACTION_DATE, pay.min_created_at) AS daySinceLoanTaken
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" t
join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_INVITATIONS a
on t.business_id = a.business_id
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_BUSINESSES" biz
on a.business_id = biz.business_id
left join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS b 
on a.business_id=json_extract_path_text(b.predict_meta, 'business_id')
join (select LENDING_BUSINESS_ID, min(created_at) as min_created_at, min(due_date) as first_due_date from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES"
group by 1) pay
on b.lending_business_id = pay.lending_business_id
where (a.business_id != '' and a.business_id is not null)
and  t.status = 'active'  AND  daySinceLoanTaken <=180 AND daySinceLoanTaken > 0
),
cal_dates_1 AS (
			SELECT DATEADD(DAY, SEQ4(), '2021-06-01') AS CAL_DATE
			 FROM TABLE(GENERATOR(ROWCOUNT=>640)) 
			 ORDER BY CAL_DATE DESC),
biz_dates_1 AS (
			SELECT a.BUSINESS_ID, MIN(a.transaction_date) AS FIRST_TXN_DATE
			from FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS"  a 
			GROUP BY a.BUSINESS_ID),
PILOT_BIZ_DATES_TEMP AS (
		SELECT *
		FROM  cal_dates_1 A LEFT JOIN
		biz_dates_1 B ON
		date(A.CAL_DATE) >= date(B.FIRST_TXN_DATE)
		ORDER BY CAL_DATE ASC)
,PILOT_BIZ_TXN_TEMP AS (
		SELECT   Business_id ,TRANSACTION_DATE,running_balance
		FROM
		        (SELECT *, RANK () OVER (PARTITION BY BUSINESS_ID,TRANSACTION_DATE ORDER BY TRANSACTION_DATE DESC) RANKS
		         FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" 
                 where 1=1
                 and status='active'
		         )
		WHERE RANKS=1
		order by Business_id, TRANSACTION_DATE)
,PILOT_DAILY_BALANCES_1  AS (
		SELECT BUSINESS_ID,FIRST_TXN_DATE, CAL_DATE,TRANSACTION_DATE,RUNNING_BALANCE,
		first_value(RUNNING_BALANCE) OVER (PARTITION BY BUSINESS_ID,GROUPER ORDER BY cal_date asc, TRANSACTION_DATE desc, RUNNING_BALANCE asc nulls last) as RUNNING_BALANCE_2
		FROM (
		SELECT A.BUSINESS_ID,A.CAL_DATE,A.FIRST_TXN_DATE,
		        B.TRANSACTION_DATE,B.RUNNING_BALANCE,
		         COUNT(running_balance) OVER (PARTITION BY a.BUSINESS_ID ORDER BY cal_date asc) as grouper
		        FROM PILOT_BIZ_DATES_TEMP A LEFT JOIN
		        PILOT_BIZ_TXN_TEMP B ON
		        A.BUSINESS_ID=B.BUSINESS_ID AND
		        date(A.CAL_DATE)= date(B.TRANSACTION_DATE)
		        ORDER BY A.CAL_DATE ASC)
		ORDER BY BUSINESS_ID,CAL_DATE)
,final_daily_running_balance AS (
		 SELECT distinct a.business_id, datediff(day, a.cal_date, b.min_created_at) as daySinceLoanTaken, cal_date, running_balance_2 AS running_balance_daily
		           FROM  PILOT_DAILY_BALANCES_1  a join past_6_mo_txn b 
		           on a.business_id = b.business_id
		          WHERE daySinceLoanTaken > 0 AND daySinceLoanTaken <= 180
		ORDER BY a.business_id, daySinceLoanTaken)
,oneMonth_rb AS (	
				SELECT business_id, min(cal_date) as min_txn_date_1mo_rb, max(cal_date) as max_txn_date_1mo_rb,
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_1m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_1m,
				               AVG(running_balance_daily) as Avg_running_balance_1M,
  							   Median(running_balance_daily) as Median_running_balance_1M,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_1M
				FROM final_daily_running_balance
				WHERE daySinceLoanTaken <=30
				GROUP BY business_id),
threeMonths_rb AS (	
				SELECT business_id, min(cal_date) as min_txn_date_3mo_rb, max(cal_date) as max_txn_date_3mo_rb,
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_3m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_3m,
				               AVG(running_balance_daily) as Avg_running_balance_3M,
  							   Median(running_balance_daily) as Median_running_balance_3M,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_3M
				FROM final_daily_running_balance
				WHERE daySinceLoanTaken <=90
				GROUP BY business_id),
 sixMonths_rb AS (	
				SELECT business_id, min(cal_date) as min_txn_date_6mo_rb, max(cal_date) as max_txn_date_6mo_rb,
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_6m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_6m,
				               AVG(running_balance_daily) as Avg_running_balance_6M,
  							   Median(running_balance_daily) as Median_running_balance_6M,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_6M
				FROM final_daily_running_balance
				WHERE daySinceLoanTaken <=180
				GROUP BY business_id),
aggregate_1_3_6_rb as (
SELECT business_id, zero_balance_count_1m, od_count_3m, COALESCE(Median_running_balance_6M, 0) as Median_running_balance_6M, 
       max_txn_date_6mo_rb, min_txn_date_6mo_rb, max_txn_date_3mo_rb, min_txn_date_3mo_rb, max_txn_date_1mo_rb, min_txn_date_1mo_rb,
       STDDEV_RUNNING_BALANCE_6M
FROM
sixMonths_rb AS a
FULL OUTER JOIN 
threeMonths_rb AS b
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_rb AS c
USING (BUSINESS_ID)
),
past_3_mo_txn AS (
SELECT * FROM past_6_mo_txn WHERE daySinceLoanTaken <= 90
),
past_1mo_txn as(
SELECT * FROM PAST_6_MO_txn WHERE daySinceLoanTaken <= 30
),
past_6_mo_agg AS (
SELECT BUSINESS_ID, min(transaction_date) as min_txn_date_6mo, max(transaction_date) as max_txn_date_6mo,
			  sum(CASE WHEN ach_credit_txn=1 THEN abs(amount) ELSE 0 end) AS ach_credit_amt_180,
			  sum(CASE WHEN ach_debit_txn=1 THEN abs(amount) ELSE 0 end) AS ach_debit_amt_180,
			  sum(CASE WHEN ach_credit_txn=1 AND abs(amount) > 100 THEN 1 ELSE 0 end) AS distinct_ach_c_txns_100_6m,
			  COALESCE(median(CASE WHEN ach_debit_txn=1 THEN abs(amount) ELSE NULL end), 0) AS median_amount_ach_d_6m,
			  COALESCE(sum(CASE WHEN ach_debit_txn=1 THEN abs(amount) ELSE 0 end) / NULLIF(sum(CASE WHEN ach_credit_txn=1 THEN abs(amount) ELSE 0 end), 0), 0) AS ratio_ach_amt_debit_credit_6m,
              COALESCE(stddev(CASE WHEN ach_credit_txn=1 THEN abs(amount) ELSE null end), 0) AS stddev_amount_ach_c_6m,
              COALESCE(sum(ach_debit_txn) / NULLIF(sum(ach_credit_txn), 0), 0) AS ratio_ach_freq_debit_credit_6m,
              COALESCE(sum(CASE WHEN card_withdrawal_txn=1 THEN abs(amount) ELSE 0 end), 0) as card_withdrawal_amt_180,
              COALESCE(sum(CASE WHEN TYPE='debit' THEN 1 ELSE 0 end) /  NULLIF(sum(CASE WHEN TYPE='credit' THEN 1 ELSE 0 end), 0), 0) as ratio_total_freq_debit_credit_6m
FROM past_6_mo_txn GROUP BY BUSINESS_ID 
),
past_3_mo_agg AS (
SELECT BUSINESS_ID, min(transaction_date) as min_txn_date_3mo, max(transaction_date) as max_txn_date_3mo,
              sum(CASE WHEN ach_credit_txn=1 THEN abs(amount) ELSE 0 end) AS ach_credit_amt_90,
			  sum(CASE WHEN ach_debit_txn=1 THEN abs(amount) ELSE 0 end) AS ach_debit_amt_90,
              COALESCE(sum(CASE WHEN TYPE='debit' THEN abs(amount) ELSE 0 end) / NULLIF(sum(CASE WHEN TYPE='credit' THEN abs(amount) ELSE 0 end), 0), 0) AS ratio_debit_credit_3m,
              COALESCE(median(CASE WHEN TYPE='debit' THEN abs(amount) ELSE null end), 0) as median_amount_debited_3m,
              COALESCE(sum(CASE WHEN card_withdrawal_txn=1 THEN abs(amount) ELSE 0 end), 0) AS card_withdrawal_amt_90,
              COALESCE(median(CASE WHEN TYPE='credit' THEN abs(amount) ELSE null end), 0) as median_amount_credited_3m,
              COALESCE(sum(ach_debit_txn), 0) as distinct_ach_d_txns_3m
FROM past_3_mo_txn GROUP BY BUSINESS_ID 
),
past_1_mo_agg AS (
SELECT BUSINESS_ID, min(transaction_date) as min_txn_date_1mo, max(transaction_date) as max_txn_date_1mo,
               STDDEV(CASE WHEN ach_credit_txn=1 THEN abs(AMOUNT) ELSE null end) AS stddev_amount_ach_c_1m,
               sum(mrdc_credit_txn) AS distinct_mrdc_txns_1m,
               COALESCE(sum(ach_debit_txn) / NULLIF(sum(ach_credit_txn), 0), 0) as ratio_ach_freq_debit_credit_1m,
               COALESCE(sum(CASE WHEN TYPE='debit' THEN abs(amount) ELSE 0 end) / NULLIF(sum(CASE WHEN TYPE='credit' THEN abs(amount) ELSE 0 end), 0), 0) AS ratio_debit_credit_1m,
               COALESCE(sum(CASE WHEN TYPE='debit' THEN abs(amount) ELSE null end), 0) as stddev_amount_debited_1m,
               COALESCE(sum(ach_debit_txn), 0) as distinct_ach_d_txns_1m,
               COALESCE(sum(CASE WHEN ach_debit_txn=1 THEN abs(amount) ELSE 0 end) / NULLIF(sum(CASE WHEN ach_credit_txn=1 THEN abs(amount) ELSE 0 end), 0), 0) AS ratio_ach_amt_debit_credit_1m,
               COALESCE(sum(CASE WHEN card_withdrawal_txn=1 and abs(amount) > 100 THEN 1 ELSE 0 end), 0) AS distinct_card_txns_100_1m
               FROM past_1mo_txn
               GROUP BY business_id
),

business_info AS (
SELECT b.BUSINESS_ID, NUMBER_OF_EMPLOYEES::int as NUMBER_OF_EMPLOYEES, 
               datediff(MONTH, to_date(DATE_OF_ESTABLISHMENT, 'YYYY-MM'), t.min_created_at) AS business_age_months, 
               datediff(month, ACCOUNT_create_date, t.min_created_at) AS months_on_book
FROM  PROD_DB.DATA.businesses b 
      LEFT JOIN PROD_DB.DATA.APPLICATIONS c ON c.application_id = b.application_id
      join past_6_mo_txn t on b.BUSINESS_ID = t.BUSINESS_ID
)
		
SELECT BUSINESS_ID, od_count_3m, zero_balance_count_1m, 
			   COALESCE(ach_credit_amt_90/ NULLIF(ach_credit_amt_180, 0), 0) AS ratio_ach_credit_amt_90_180,
			   COALESCE(ach_debit_amt_90/ NULLIF(ach_debit_amt_180, 0), 0) AS ratio_ach_debit_amt_90_180, 
			   COALESCE(stddev_amount_ach_c_1m, 0) as stddev_amount_ach_c_1m, COALESCE(distinct_ach_c_txns_100_6m, 0) as distinct_ach_c_txns_100_6m, 
			   COALESCE(median_amount_ach_d_6m, 0) as median_amount_ach_d_6m, 
			   COALESCE(distinct_mrdc_txns_1m, 0) as distinct_mrdc_txns_1m,
			   COALESCE(ratio_ach_amt_debit_credit_6m, 0) as ratio_ach_amt_debit_credit_6m, COALESCE(ratio_ach_freq_debit_credit_1m, 0) as ratio_ach_freq_debit_credit_1m, 
			   COALESCE(ratio_debit_credit_1m, 0) as ratio_debit_credit_1m, COALESCE(ratio_debit_credit_3m, 0) as ratio_debit_credit_3m, 
			   median_running_balance_6m, median_amount_debited_3m, stddev_amount_debited_1m, stddev_amount_ach_c_6m, distinct_ach_d_txns_1m,
               ratio_ach_amt_debit_credit_1m, ratio_ach_freq_debit_credit_6m, 
               COALESCE(card_withdrawal_amt_90/ NULLIF(card_withdrawal_amt_180, 0), 0) AS ratio_card_withdrawal_amt_90_180,
               median_amount_credited_3m, distinct_ach_d_txns_3m, distinct_card_txns_100_1m,
               ratio_total_freq_debit_credit_6m, STDDEV_RUNNING_BALANCE_6M, 
               months_on_book, business_age_months, number_of_employees
FROM aggregate_1_3_6_rb 
	LEFT JOIN  past_6_mo_agg using(business_id)
	LEFT JOIN past_3_mo_agg using(business_id)
	LEFT JOIN past_1_mo_agg using(business_id)
    LEFT JOIN business_info using(business_id) 
"""

In [7]:
querySno = SQLQuery('snowflake')
ldf = querySno(lending_features)

ValueError: unsupported format character 'A' (0x41) at index 299

In [10]:
querySno = SQLQuery('snowflake')
ldf = querySno(r"""
WITH past_6_mo_txn as (
  		SELECT t.id AS transaction_id, t.BUSINESS_ID , amount, TYPE, TRANSACTION_DATE,  t.status, running_balance,
        pay.lending_business_id, pay.min_created_at, pay.first_due_date,
              CASE WHEN TYPE = 'credit' and DESCRIPTION  ILIKE ANY(
	                    '%AMAZON%'
	                    , '%AMZN%'
	                    , '%STRIPE%'
	                    , '%SQUARE INC%'
	                    , '%SHOPIFY%'
	                    , '%SHOPPAY%'
	                    )
                    AND NOT(
	                    description ILIKE ANY(
	                        '%LYFT'
	                        , '%OfferUp%'
	                        , '%Gumroad%'
	                        , '%FB%Fundrai%'
	                        , '%Verify%'
	                        , '%CASH%'
	                        , '%PAYROLL%'
	                        , '%VRFY%'
	                        , '%CAPITAL%'
	                        , '%REFUND%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
                CASE WHEN TYPE ='debit' AND MEDIUM  ILIKE ANY('%external withdrawal%', '%iat withdrawal%') THEN 1 ELSE 0 END AS ach_debit_txn,
                CASE WHEN type = 'credit'  AND medium ILIKE ANY('%external deposit%', '%iat deposit%') THEN 1 ELSE 0 END AS ach_credit_txn,
                CASE WHEN medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
                CASE WHEN TYPE = 'credit' AND description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,
                datediff(day, TRANSACTION_DATE, pay.min_created_at) AS daySinceLoanTaken
FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" t
join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_INVITATIONS a
on t.business_id = a.business_id
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_BUSINESSES" biz
on a.business_id = biz.business_id
left join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS b 
on a.business_id=json_extract_path_text(b.predict_meta, 'business_id')
join (select LENDING_BUSINESS_ID, min(created_at) as min_created_at, min(due_date) as first_due_date from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES"
group by 1) pay
on b.lending_business_id = pay.lending_business_id
where (a.business_id != '' and a.business_id is not null)
and  t.status = 'active'  AND  daySinceLoanTaken <=180 AND daySinceLoanTaken > 0
),
cal_dates_1 AS (
			SELECT DATEADD(DAY, SEQ4(), '2021-06-01') AS CAL_DATE
			 FROM TABLE(GENERATOR(ROWCOUNT=>640)) 
			 ORDER BY CAL_DATE DESC),
biz_dates_1 AS (
			SELECT a.BUSINESS_ID, MIN(a.transaction_date) AS FIRST_TXN_DATE
			from FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS"  a 
			GROUP BY a.BUSINESS_ID),
PILOT_BIZ_DATES_TEMP AS (
		SELECT *
		FROM  cal_dates_1 A LEFT JOIN
		biz_dates_1 B ON
		date(A.CAL_DATE) >= date(B.FIRST_TXN_DATE)
		ORDER BY CAL_DATE ASC)
,PILOT_BIZ_TXN_TEMP AS (
		SELECT   Business_id ,TRANSACTION_DATE,running_balance
		FROM
		        (SELECT *, RANK () OVER (PARTITION BY BUSINESS_ID,TRANSACTION_DATE ORDER BY TRANSACTION_DATE DESC) RANKS
		         FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" 
                 where 1=1
                 and status='active'
		         )
		WHERE RANKS=1
		order by Business_id, TRANSACTION_DATE)
,PILOT_DAILY_BALANCES_1  AS (
		SELECT BUSINESS_ID,FIRST_TXN_DATE, CAL_DATE,TRANSACTION_DATE,RUNNING_BALANCE,
		first_value(RUNNING_BALANCE) OVER (PARTITION BY BUSINESS_ID,GROUPER ORDER BY cal_date asc, TRANSACTION_DATE desc, RUNNING_BALANCE asc nulls last) as RUNNING_BALANCE_2
		FROM (
		SELECT A.BUSINESS_ID,A.CAL_DATE,A.FIRST_TXN_DATE,
		        B.TRANSACTION_DATE,B.RUNNING_BALANCE,
		         COUNT(running_balance) OVER (PARTITION BY a.BUSINESS_ID ORDER BY cal_date asc) as grouper
		        FROM PILOT_BIZ_DATES_TEMP A LEFT JOIN
		        PILOT_BIZ_TXN_TEMP B ON
		        A.BUSINESS_ID=B.BUSINESS_ID AND
		        date(A.CAL_DATE)= date(B.TRANSACTION_DATE)
		        ORDER BY A.CAL_DATE ASC)
		ORDER BY BUSINESS_ID,CAL_DATE)
,final_daily_running_balance AS (
		 SELECT distinct a.business_id, datediff(day, a.cal_date, b.min_created_at) as daySinceLoanTaken, cal_date, running_balance_2 AS running_balance_daily
		           FROM  PILOT_DAILY_BALANCES_1  a join past_6_mo_txn b 
		           on a.business_id = b.business_id
		          WHERE daySinceLoanTaken > 0 AND daySinceLoanTaken <= 180
		ORDER BY a.business_id, daySinceLoanTaken)
,oneMonth_rb AS (	
				SELECT business_id, min(cal_date) as min_txn_date_1mo_rb, max(cal_date) as max_txn_date_1mo_rb,
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_1m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_1m,
				               AVG(running_balance_daily) as Avg_running_balance_1M,
  							   Median(running_balance_daily) as Median_running_balance_1M,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_1M
				FROM final_daily_running_balance
				WHERE daySinceLoanTaken <=30
				GROUP BY business_id),
threeMonths_rb AS (	
				SELECT business_id, min(cal_date) as min_txn_date_3mo_rb, max(cal_date) as max_txn_date_3mo_rb,
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_3m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_3m,
				               AVG(running_balance_daily) as Avg_running_balance_3M,
  							   Median(running_balance_daily) as Median_running_balance_3M,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_3M
				FROM final_daily_running_balance
				WHERE daySinceLoanTaken <=90
				GROUP BY business_id),
 sixMonths_rb AS (	
				SELECT business_id, min(cal_date) as min_txn_date_6mo_rb, max(cal_date) as max_txn_date_6mo_rb,
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_6m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_6m,
				               AVG(running_balance_daily) as Avg_running_balance_6M,
  							   Median(running_balance_daily) as Median_running_balance_6M,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_6M
				FROM final_daily_running_balance
				WHERE daySinceLoanTaken <=180
				GROUP BY business_id),
aggregate_1_3_6_rb as (
SELECT business_id, zero_balance_count_1m, od_count_3m, COALESCE(Median_running_balance_6M, 0) as Median_running_balance_6M, 
       max_txn_date_6mo_rb, min_txn_date_6mo_rb, max_txn_date_3mo_rb, min_txn_date_3mo_rb, max_txn_date_1mo_rb, min_txn_date_1mo_rb,
       STDDEV_RUNNING_BALANCE_6M
FROM
sixMonths_rb AS a
FULL OUTER JOIN 
threeMonths_rb AS b
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_rb AS c
USING (BUSINESS_ID)
),
past_3_mo_txn AS (
SELECT * FROM past_6_mo_txn WHERE daySinceLoanTaken <= 90
),
past_1mo_txn as(
SELECT * FROM PAST_6_MO_txn WHERE daySinceLoanTaken <= 30
),
past_6_mo_agg AS (
SELECT BUSINESS_ID, min(transaction_date) as min_txn_date_6mo, max(transaction_date) as max_txn_date_6mo,
			  sum(CASE WHEN ach_credit_txn=1 THEN abs(amount) ELSE 0 end) AS ach_credit_amt_180,
			  sum(CASE WHEN ach_debit_txn=1 THEN abs(amount) ELSE 0 end) AS ach_debit_amt_180,
			  sum(CASE WHEN ach_credit_txn=1 AND abs(amount) > 100 THEN 1 ELSE 0 end) AS distinct_ach_c_txns_100_6m,
			  COALESCE(median(CASE WHEN ach_debit_txn=1 THEN abs(amount) ELSE NULL end), 0) AS median_amount_ach_d_6m,
			  COALESCE(sum(CASE WHEN ach_debit_txn=1 THEN abs(amount) ELSE 0 end) / NULLIF(sum(CASE WHEN ach_credit_txn=1 THEN abs(amount) ELSE 0 end), 0), 0) AS ratio_ach_amt_debit_credit_6m,
              COALESCE(stddev(CASE WHEN ach_credit_txn=1 THEN abs(amount) ELSE null end), 0) AS stddev_amount_ach_c_6m,
              COALESCE(sum(ach_debit_txn) / NULLIF(sum(ach_credit_txn), 0), 0) AS ratio_ach_freq_debit_credit_6m,
              COALESCE(sum(CASE WHEN card_withdrawal_txn=1 THEN abs(amount) ELSE 0 end), 0) as card_withdrawal_amt_180,
              COALESCE(sum(CASE WHEN TYPE='debit' THEN 1 ELSE 0 end) /  NULLIF(sum(CASE WHEN TYPE='credit' THEN 1 ELSE 0 end), 0), 0) as ratio_total_freq_debit_credit_6m
FROM past_6_mo_txn GROUP BY BUSINESS_ID 
),
past_3_mo_agg AS (
SELECT BUSINESS_ID, min(transaction_date) as min_txn_date_3mo, max(transaction_date) as max_txn_date_3mo,
              sum(CASE WHEN ach_credit_txn=1 THEN abs(amount) ELSE 0 end) AS ach_credit_amt_90,
			  sum(CASE WHEN ach_debit_txn=1 THEN abs(amount) ELSE 0 end) AS ach_debit_amt_90,
              COALESCE(sum(CASE WHEN TYPE='debit' THEN abs(amount) ELSE 0 end) / NULLIF(sum(CASE WHEN TYPE='credit' THEN abs(amount) ELSE 0 end), 0), 0) AS ratio_debit_credit_3m,
              COALESCE(median(CASE WHEN TYPE='debit' THEN abs(amount) ELSE null end), 0) as median_amount_debited_3m,
              COALESCE(sum(CASE WHEN card_withdrawal_txn=1 THEN abs(amount) ELSE 0 end), 0) AS card_withdrawal_amt_90,
              COALESCE(median(CASE WHEN TYPE='credit' THEN abs(amount) ELSE null end), 0) as median_amount_credited_3m,
              COALESCE(sum(ach_debit_txn), 0) as distinct_ach_d_txns_3m
FROM past_3_mo_txn GROUP BY BUSINESS_ID 
),
past_1_mo_agg AS (
SELECT BUSINESS_ID, min(transaction_date) as min_txn_date_1mo, max(transaction_date) as max_txn_date_1mo,
               STDDEV(CASE WHEN ach_credit_txn=1 THEN abs(AMOUNT) ELSE null end) AS stddev_amount_ach_c_1m,
               sum(mrdc_credit_txn) AS distinct_mrdc_txns_1m,
               COALESCE(sum(ach_debit_txn) / NULLIF(sum(ach_credit_txn), 0), 0) as ratio_ach_freq_debit_credit_1m,
               COALESCE(sum(CASE WHEN TYPE='debit' THEN abs(amount) ELSE 0 end) / NULLIF(sum(CASE WHEN TYPE='credit' THEN abs(amount) ELSE 0 end), 0), 0) AS ratio_debit_credit_1m,
               COALESCE(sum(CASE WHEN TYPE='debit' THEN abs(amount) ELSE null end), 0) as stddev_amount_debited_1m,
               COALESCE(sum(ach_debit_txn), 0) as distinct_ach_d_txns_1m,
               COALESCE(sum(CASE WHEN ach_debit_txn=1 THEN abs(amount) ELSE 0 end) / NULLIF(sum(CASE WHEN ach_credit_txn=1 THEN abs(amount) ELSE 0 end), 0), 0) AS ratio_ach_amt_debit_credit_1m,
               COALESCE(sum(CASE WHEN card_withdrawal_txn=1 and abs(amount) > 100 THEN 1 ELSE 0 end), 0) AS distinct_card_txns_100_1m
               FROM past_1mo_txn
               GROUP BY business_id
),

business_info AS (
SELECT b.BUSINESS_ID, NUMBER_OF_EMPLOYEES::int as NUMBER_OF_EMPLOYEES, 
               datediff(MONTH, to_date(DATE_OF_ESTABLISHMENT, 'YYYY-MM'), t.min_created_at) AS business_age_months, 
               datediff(month, ACCOUNT_create_date, t.min_created_at) AS months_on_book
FROM  PROD_DB.DATA.businesses b 
      LEFT JOIN PROD_DB.DATA.APPLICATIONS c ON c.application_id = b.application_id
      join past_6_mo_txn t on b.BUSINESS_ID = t.BUSINESS_ID
)
		
SELECT BUSINESS_ID, od_count_3m, zero_balance_count_1m, 
			   COALESCE(ach_credit_amt_90/ NULLIF(ach_credit_amt_180, 0), 0) AS ratio_ach_credit_amt_90_180,
			   COALESCE(ach_debit_amt_90/ NULLIF(ach_debit_amt_180, 0), 0) AS ratio_ach_debit_amt_90_180, 
			   COALESCE(stddev_amount_ach_c_1m, 0) as stddev_amount_ach_c_1m, COALESCE(distinct_ach_c_txns_100_6m, 0) as distinct_ach_c_txns_100_6m, 
			   COALESCE(median_amount_ach_d_6m, 0) as median_amount_ach_d_6m, 
			   COALESCE(distinct_mrdc_txns_1m, 0) as distinct_mrdc_txns_1m,
			   COALESCE(ratio_ach_amt_debit_credit_6m, 0) as ratio_ach_amt_debit_credit_6m, COALESCE(ratio_ach_freq_debit_credit_1m, 0) as ratio_ach_freq_debit_credit_1m, 
			   COALESCE(ratio_debit_credit_1m, 0) as ratio_debit_credit_1m, COALESCE(ratio_debit_credit_3m, 0) as ratio_debit_credit_3m, 
			   median_running_balance_6m, median_amount_debited_3m, stddev_amount_debited_1m, stddev_amount_ach_c_6m, distinct_ach_d_txns_1m,
               ratio_ach_amt_debit_credit_1m, ratio_ach_freq_debit_credit_6m, 
               COALESCE(card_withdrawal_amt_90/ NULLIF(card_withdrawal_amt_180, 0), 0) AS ratio_card_withdrawal_amt_90_180,
               median_amount_credited_3m, distinct_ach_d_txns_3m, distinct_card_txns_100_1m,
               ratio_total_freq_debit_credit_6m, STDDEV_RUNNING_BALANCE_6M, 
               months_on_book, business_age_months, number_of_employees
FROM aggregate_1_3_6_rb 
	LEFT JOIN  past_6_mo_agg using(business_id)
	LEFT JOIN past_3_mo_agg using(business_id)
	LEFT JOIN past_1_mo_agg using(business_id)
    LEFT JOIN business_info using(business_id) 
"""
)

ValueError: unsupported format character 'A' (0x41) at index 299